In [1]:
import re
import tabula
import requests
import json
import csv
import boto3
import pandas as pd
import numpy as np
from data_extraction import DataExtractor as de
from database_utils import DatabaseConnector as dc
from datetime import datetime as dt

extr = de()
conn = dc()

In [6]:
extr = de()
conn = dc()
df = extr.retrieve_product_data()
df['weight_check'] = df['weight'].str.contains('x')
dg = df['weight'][df['weight_check'] == True].str.replace('[xg]', '', regex=True).str.split(expand=True)
dg['weight'] = dg[0].astype(int) * dg[1].astype(int)
dg['weight'] = dg['weight'].astype(str) + 'g'
df.update(dg)
df = df.drop(['weight_check'], axis=1)
grams = df[df.loc[:, 'weight'].str.contains('([^k][g])|[ml]') == True]
grams.loc[:, 'weight'] = grams.loc[:, 'weight'].str.removesuffix('g')
grams.loc[:, 'weight'] = grams.loc[:, 'weight'].str.removesuffix('g .')
grams.loc[:, 'weight'] = grams.loc[:, 'weight'].str.removesuffix('ml')
grams.loc[:, 'weight'] = pd.to_numeric(grams.loc[:, 'weight']) / 1000
#print(grams.loc[2, 'weight'], type(grams.loc[2, 'weight']))
df.update(grams)
ounces = df[df.loc[:, 'weight'].str.contains('oz') == True]
ounces.loc[:, 'weight'] = ounces.loc[:, 'weight'].str.removesuffix('oz')
ounces.loc[:, 'weight'] = round(ounces.loc[:, 'weight'].astype(float) / 35.274, 2)
df.update(ounces)
df.loc[:, 'weight'] = df.loc[:, 'weight'].str.removesuffix('kg')
df.rename(columns={'weight': 'weight (kg)'}, inplace=True)

C:\Users\srayn\AppData\Local\Temp\ipykernel_5028\4032240571.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  grams = df[df.loc[:, 'weight'].str.contains('([^k][g])|[ml]') == True]
C:\Users\srayn\AppData\Local\Temp\ipykernel_5028\4032240571.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grams.loc[:, 'weight'] = grams.loc[:, 'weight'].str.removesuffix('g')
C:\Users\srayn\AppData\Local\Temp\ipykernel_5028\4032240571.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [39]:
df = df.drop('Unnamed: 0', axis=1)
df = df[df['removed'].isin(['Still_avaliable', 'Removed']) == True]
df.loc[:, 'product_price'] = df.loc[:, 'product_price'].str.replace('£', '')
df.rename(columns={'product_price': 'product_price (£)'}, inplace=True)
df['date_added'] = pd.to_datetime(df['date_added']).dt.date
df['product_price (£)'] = df['product_price (£)'].astype(float)
df['weight (kg)'] = df['weight (kg)'].astype(float)
df.reset_index(drop=True, inplace=True)

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     1853 non-null   float64
 1   product_name   1849 non-null   object 
 2   product_price  1849 non-null   object 
 3   weight (kg)    957 non-null    object 
 4   category       1849 non-null   object 
 5   EAN            1849 non-null   object 
 6   date_added     1849 non-null   object 
 7   uuid           1849 non-null   object 
 8   removed        1849 non-null   object 
 9   product_code   1849 non-null   object 
dtypes: float64(1), object(9)
memory usage: 144.9+ KB
None
